In [17]:
import pandas as pd
pd.options.display.max_columns = 100
import numpy as np
from pandarallel import pandarallel

import scipy.optimize as opt

from pathlib import Path
import pathlib
from tqdm.notebook import tqdm
INPUT = Path("../input/google-smartphone-decimeter-challenge/")

In [18]:
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
      np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

# Apply WLS on one collection and one measurement

In [20]:
# directory setting
INPUT = '../input/google-smartphone-decimeter-challenge'
p = pathlib.Path(INPUT)

base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')

# test derived
test_files = list(p.glob('test/*/*/*_derived.csv'))
print('test_derived.csv count :', len(test_files))

tds = []
for test_file in tqdm(test_files):
    tds.append(pd.read_csv(test_file))
test_derived = pd.concat(tds)
test_derived['phone'] = test_derived['collectionName'] + '_' + test_derived['phoneName']


collection_name = [i for i in base_test['phone'].unique()]
phone = [i for i in base_test['phoneName'].unique()]

# base_testに含まれているmillisSinceGpsEpochを使う
test_derived = test_derived[test_derived['millisSinceGpsEpoch'].isin(base_test['millisSinceGpsEpoch'])]
# derivedのサンプルが少ないと、最適化がうまくできないので、20以上のものだけを使う
valid_index = test_derived['millisSinceGpsEpoch'].value_counts()[test_derived['millisSinceGpsEpoch'].value_counts()>20].index
test_derived = test_derived[test_derived['millisSinceGpsEpoch'].isin(valid_index)]


# Corrected pseudorange according to data instructions
test_derived['correctedPrM'] = test_derived.apply(
    lambda r: r.rawPrM + r.satClkBiasM - r.isrbM - r.ionoDelayM - r.tropoDelayM,
    axis=1
).copy()

# Time it took for signal to travel
light_speed = 299_792_458
test_derived['transmissionTimeSeconds'] = (test_derived['correctedPrM'] / light_speed).copy()


# Compute true sat positions at arrival time
omega_e = 7.2921151467e-5
test_derived['xSatPosMRotated'] = \
    np.cos(omega_e * test_derived['transmissionTimeSeconds']) * test_derived['xSatPosM'] \
    + np.sin(omega_e * test_derived['transmissionTimeSeconds']) * test_derived['ySatPosM']
    
test_derived['ySatPosMRotated'] = \
    - np.sin(omega_e * test_derived['transmissionTimeSeconds']) * test_derived['xSatPosM'] \
    + np.cos(omega_e * test_derived['transmissionTimeSeconds']) * test_derived['ySatPosM']
    
test_derived['zSatPosMRotated'] = test_derived['zSatPosM']


# Uncertainty weight for the WLS method
test_derived['uncertaintyWeight'] = 1 / test_derived['rawPrUncM']


def apply_wls_pos(df):
    '''
    input: millisSinceGpsEpochごとのデータフレーム
    output: 修正済みlat, lng
    '''
    
    def distance(sat_pos, x):
        '''
        input: millisSinceGpsEpochごとにまとめられたデータフレーム
        output: 入力されたデータフレームの行数と同じ大きさのベクトル
        '''
        sat_pos_diff = sat_pos.copy(deep=True)
        sat_pos_diff['xSatPosMRotated'] = sat_pos_diff['xSatPosMRotated'] - x[0]
        sat_pos_diff['ySatPosMRotated'] = sat_pos_diff['ySatPosMRotated'] - x[1]
        sat_pos_diff['zSatPosMRotated'] = sat_pos_diff['zSatPosMRotated'] - x[2]

        sat_pos_diff['d'] = sat_pos_diff.apply(
            lambda r: r.uncertaintyWeight * 
                (np.sqrt((r.xSatPosMRotated**2 + r.ySatPosMRotated**2 + r.zSatPosMRotated**2)) + x[3] - r.correctedPrM),
            axis=1
        )
        return sat_pos_diff['d'].values

    def distance_fixed_satpos(x):
        return distance(df[['xSatPosMRotated', 'ySatPosMRotated', 'zSatPosMRotated', 'correctedPrM', 'uncertaintyWeight']], x)


    # Start point for the optimiser
    x0= [0,0,0,0]

    opt_res = opt.least_squares(distance_fixed_satpos, x0)

    # Optimiser yields a position in the ECEF coordinates
    opt_res_pos = opt_res.x

    # ECEF position to lat/long
    wls_estimated_pos = ecef2lla(*opt_res_pos[:3]) # x,y,z
    wls_estimated_pos = np.squeeze(wls_estimated_pos) # １次元を削除
    
    
    return [wls_estimated_pos[0], wls_estimated_pos[1]]

test_derived.csv count : 48


  0%|          | 0/48 [00:00<?, ?it/s]

In [21]:
%%time
pandarallel.initialize(use_memory_fs=False)
result = test_derived.groupby('millisSinceGpsEpoch').parallel_apply(apply_wls_pos)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
CPU times: user 18 s, sys: 3.68 s, total: 21.7 s
Wall time: 1h 50min 59s


## 整形

In [22]:
# 修正済みlatDeg, lngDegを取り出す
fixed_lat = []
fixed_lng = []
for i in result.values:
    fixed_lat.append(i[0])
    fixed_lng.append(i[1])
result_df = pd.DataFrame({'fixed_latDeg':fixed_lat, 'fixed_lngDeg':fixed_lng}, index=result.index)

# 修正済みlatDeg, lngDegをbase_testに適用させる
fixed_base_test = pd.merge(base_test, result_df, on='millisSinceGpsEpoch', how='left')
fixed_base_test.loc[fixed_base_test['fixed_latDeg'].isnull(), 'fixed_latDeg'] = fixed_base_test['latDeg']
fixed_base_test.loc[fixed_base_test['fixed_lngDeg'].isnull(), 'fixed_lngDeg'] = fixed_base_test['lngDeg']
fixed_base_test = fixed_base_test.drop(['latDeg', 'lngDeg'], axis=1)
fixed_base_test = fixed_base_test.rename(columns={'fixed_latDeg':'latDeg',
                                                    'fixed_lngDeg':'lngDeg'})
fixed_base_test = fixed_base_test.reindex(columns=['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM', 'phone'])
fixed_base_test.head()

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone
0,2020-05-15-US-MTV-1,Pixel4,1273608785432,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Pixel4
1,2020-05-15-US-MTV-1,Pixel4,1273608786432,37.416646,-122.082040,-31.76,2020-05-15-US-MTV-1_Pixel4
2,2020-05-15-US-MTV-1,Pixel4,1273608787432,37.416652,-122.082039,-31.65,2020-05-15-US-MTV-1_Pixel4
3,2020-05-15-US-MTV-1,Pixel4,1273608788432,37.416607,-122.082063,-31.52,2020-05-15-US-MTV-1_Pixel4
4,2020-05-15-US-MTV-1,Pixel4,1273608789432,37.416609,-122.082073,-28.95,2020-05-15-US-MTV-1_Pixel4


In [23]:
fixed_base_test.to_csv('../output/fixed_base_test.csv', index=False)